In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString, pid
from ST import printSymTab, find, find2
from HM import LinkedList

Importing Jupyter notebook from P0.ipynb
Importing Jupyter notebook from SC.ipynb
Importing Jupyter notebook from ST.ipynb
Importing Jupyter notebook from HM.ipynb
size is 10, data: None, is_alloc: False, value: None, nxt: 
size is 10, data: Var(name = x, lev = , tp = Pointer(name = , id = 0, lev = , tp = <class 'ST.Int'>)), is_alloc: True, value: None, nxt: 
None


In [2]:

def runpywasm(wasmfile):
    import pywasm
    def write(i): print(i)
    def writeln(): print('\n')
    def read(): return int(input())
    # Two StdProc for explicit allocation and deallocation
    def new(x): # x is the pid of the pointer type variable; 
        p = find2(x)
        heap.allocate(p)
    def dispose(x): # x is the pid of the pointer type variable; 
        p = find2(x)
        heap.deallocate(p)
    def set_value(value,pid): # for assignment, p^ := 4;
        p = find2(pid)
        target = heap.search(p)
        if target != None:
            target.set_value(value=value)
        else:
            print(p," is not allocated!")
    def get_value(pid): # for example, x := p^
        p = find2(pid)
        target = heap.search(p)
        if target != None:
            return target.get_value()
        else:
            print(p," is not allocated!")
            return 0
    def set_value1(value,index,pid): #for array
        p = find2(pid)
        target = heap.search(p)
        if target != None:
            target.set_value(value=value,index=index)
        else:
            print(p," is not allocated!")
    def get_value1(index,pid):
        p = find2(pid)
        target = heap.search(p)
        if target != None:
            return target.get_value(index=index)
        else:
            print(p," is not allocated!")
    def set_value2(index1,index2):
        p1 = find2(index1)
        p2 = find2(index2)
        target1 = heap.search(p1)
        target2 = heap.search(p2)
        if target1 != None and target2 != None:
            target1.value = target2.value
        else:
            print("One/Both of ",p1,' ',p2," is/are not allocated!")
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read, 'new': new, 'dispose': dispose, 'set_value': set_value, 'get_value': get_value, 'set_value1': set_value1, 'get_value1': get_value1, 'set_value2': set_value2}})

## Test Cases for explicit allocation and deallocation

In [20]:
heap = LinkedList()
compileString("""
program p;
  type oo = ^array [1..5] of array [1..5] of array [1..5] of integer;
  type oo1 = array [1..5] of array [1..3] of ^record tt : array [1..5] of integer end;
  type oo2 = array [1..5] of array [1..5] of integer;
  type oo3 = array [1..3] of ^integer;
  type i = ^integer;
  var x : oo;
  var y : oo1;
  var z : oo2;
  var p1 : oo3;
  var p2 : i;
  begin
    new(y[1][1]);
    new(y[2][1]);
    y[1][1].tt[1]^ := 3;
    y[2][1].tt[1]^  := 4;
    write(y[2][1].tt[1]^);
    write(y[1][1].tt[1]^);
    dispose(y[2][1])
  end
""")

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(func $program
i32.const 1
call $new
i32.const 4
call $new
i32.const 3
i32.const 0
i32.const 1
call $set_value1
i32.const 4
i32.const 0
i32.const 4
call $set_value1
i32.const 0
i32.const 4
call $get_value1
call $write
i32.const 0
i32.const 1
call $get_value1
call $write
i32.const 4
call $dispose
)
(memory 1)
(start $program)
)


In [21]:
%%writefile test.wat
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(func $program
i32.const 1
call $new
i32.const 4
call $new
i32.const 3
i32.const 0
i32.const 1
call $set_value1
i32.const 4
i32.const 0
i32.const 4
call $set_value1
i32.const 0
i32.const 4
call $get_value1
call $write
i32.const 0
i32.const 1
call $get_value1
call $write
i32.const 4
call $dispose
)
(memory 1)
(start $program)
)

Overwriting test.wat


In [22]:
!wat2wasm test.wat

In [23]:
runpywasm("test.wasm")

4
3


In [24]:
print(heap.start)

size is 20, data: Var(name = y, lev = , tp = Pointer(name = , id = 1, lev = -3, tp = Record(fields = [Var(name = tt, lev = 1, tp = Array(lower = 1, length = 5, base = <class 'ST.Int'>))]))), is_alloc: True, value: [3, None, None, None, None], nxt: 
size is 20, data: None, is_alloc: False, value: None, nxt: 
None


In [9]:
heap = LinkedList();
compileString("""
program p;
  type fp = record tt : array [1..5] of ^integer end;
  type test2 = array [1..3] of record tt1 : ^integer end;
  var y: fp;
  var x : test2;
  begin
    new(y.tt[1]);
    y.tt[1]^ := 5;
    new(x[2].tt1);
    x[2].tt1^ := 6;
    write(x[2].tt1^);
    dispose(y.tt[1])
  end
""")
#printSymTab()

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(func $program
i32.const 0
call $new
i32.const 5
i32.const 0
call $set_value
i32.const 6
call $new
i32.const 6
i32.const 6
call $set_value
i32.const 6
call $get_value
call $write
i32.const 0
call $dispose
)
(memory 1)
(start $program)
)


In [10]:
%%writefile test.wat
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(func $program
i32.const 0
call $new
i32.const 5
i32.const 0
call $set_value
i32.const 6
call $new
i32.const 6
i32.const 6
call $set_value
i32.const 6
call $get_value
call $write
i32.const 0
call $dispose
)
(memory 1)
(start $program)
)

Overwriting test.wat


In [11]:
!wat2wasm test.wat

In [12]:
runpywasm("test.wasm")

6


In [13]:
print(heap.start)

size is 4, data: None, is_alloc: False, value: None, nxt: 
size is 4, data: Var(name = tt1, lev = , tp = Pointer(name = , id = 6, lev = -3, tp = <class 'ST.Int'>)), is_alloc: True, value: 6, nxt: 
None


In [25]:
heap = LinkedList();
compileString("""
program p;
  type oo = ^array [1..5] of array [1..5] of array [1..5] of integer;
  type oo1 = array [1..5] of array [1..3] of array [1..2] of ^record tt : array [1..5] of integer end;
  type oo2 = array [1..5] of array [1..5] of integer;
  type oo3 = array [1..3] of ^integer;
  type i = integer;
  var x : oo;
  var y : oo1;
  var z : oo2;
  var p1 : oo3;
  var p2 : i;
  procedure q(b: boolean);
    begin b := b = false
    end;
  begin
    new(y[2][1][1]);
    new(y[1][1][1]);
    y[2][1][1].tt[1]^ := 4;
    p2 := y[2][1][1].tt[1]^;
    write(p2);
    write(y[2][1][1].tt[1]^)
  end
""")
#printSymTab()

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(global $p2 (mut i32) i32.const 0)
(func $q (param $b i32)
local.get $b
i32.const 0
i32.eq
local.set $b
)
(func $program
i32.const 7
call $new
i32.const 1
call $new
i32.const 4
i32.const 0
i32.const 7
call $set_value1
i32.const 0
i32.const 7
call $get_value1
global.set $p2
global.get $p2
call $write
i32.const 0
i32.const 7
call $get_value1
call $write
)
(me

In [26]:
%%writefile test.wat
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(global $p2 (mut i32) i32.const 0)
(func $q (param $b i32)
local.get $b
i32.const 0
i32.eq
local.set $b
)
(func $program
i32.const 7
call $new
i32.const 1
call $new
i32.const 4
i32.const 0
i32.const 7
call $set_value1
i32.const 0
i32.const 7
call $get_value1
global.set $p2
global.get $p2
call $write
i32.const 0
i32.const 7
call $get_value1
call $write
)
(memory 1)
(start $program)
)

Overwriting test.wat


In [27]:
!wat2wasm test.wat

In [28]:
runpywasm("test.wasm")

4
4


In [29]:
print(heap.start)

size is 20, data: Var(name = y, lev = , tp = Pointer(name = , id = 7, lev = -3, tp = Record(fields = [Var(name = tt, lev = 1, tp = Array(lower = 1, length = 5, base = <class 'ST.Int'>))]))), is_alloc: True, value: [4, None, None, None, None], nxt: 
size is 20, data: Var(name = y, lev = , tp = Pointer(name = , id = 1, lev = -3, tp = Record(fields = [Var(name = tt, lev = 1, tp = Array(lower = 1, length = 5, base = <class 'ST.Int'>))]))), is_alloc: True, value: [None, None, None, None, None], nxt: 
None


In [18]:
heap = LinkedList();
compileString("""
program p;
  type oo = ^array [1..5] of array [1..5] of array [1..5] of integer;
  type oo1 = ^array [1..5] of record tt : array [1..5] of integer end;
  type oo2 = array [1..5] of array [1..5] of integer;
  type oo3 = array [1..3] of ^integer;
  type i = ^integer;
  var x : oo;
  var y : oo1;
  var z : oo2;
  var p1 : oo3;
  var p2 : i;
  procedure q(b: boolean);
    begin b := b = false
    end;
  begin
    new(x);
    new(y);
    new(p2);
    z[1][1] := 5;
    write(z[1][1]);
    x[1][1][1]^ := 8;
    y[1].tt[1]^ := 16;
    write(x[1][1][1]^);
    write(y[1].tt[1]^);
    new(p1[1]);
    p2^ := 4;
    p1[1] := p2
  end
""")
#printSymTab()

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(func $q (param $b i32)
local.get $b
i32.const 0
i32.eq
local.set $b
)
(func $program
i32.const 0
call $new
i32.const 1
call $new
i32.const 5
call $new
i32.const 0
i32.const 5
i32.store
i32.const 0
i32.load
call $write
i32.const 8
i32.const 0
i32.const 0
call $set_value1
i32.const 16
i32.const 0
i32.const 1
call $set_value1
i32.const 0
i32.const 0
call $get

In [19]:
%%writefile test.wat
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(func $q (param $b i32)
local.get $b
i32.const 0
i32.eq
local.set $b
)
(func $program
i32.const 0
call $new
i32.const 1
call $new
i32.const 5
call $new
i32.const 0
i32.const 5
i32.store
i32.const 0
i32.load
call $write
i32.const 8
i32.const 0
i32.const 0
call $set_value1
i32.const 16
i32.const 0
i32.const 1
call $set_value1
i32.const 0
i32.const 0
call $get_value1
call $write
i32.const 0
i32.const 1
call $get_value1
call $write
i32.const 2
call $new
i32.const 4
i32.const 5
call $set_value
i32.const 2
i32.const 5
call $set_value2
)
(memory 1)
(start $program)
)

Overwriting test.wat


In [20]:
!wat2wasm test.wat

In [21]:
runpywasm("test.wasm")

5
8
16


In [22]:
print(heap.start)

size is 500, data: Var(name = x, lev = -3, tp = Pointer(name = , id = 0, lev = , tp = Array(lower = 1, length = 5, base = Array(lower = 1, length = 5, base = Array(lower = 1, length = 5, base = <class 'ST.Int'>))))), is_alloc: True, value: [8, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None], nxt: size i

In [23]:
heap = LinkedList();
compileString("""
program p;
  type d = array [1..5] of integer;
  type oo = array [1..5] of array [1..5] of integer;
  type oo1 = array [1..5] of record tt : integer end;
  type d3 = ^array [1..5] of integer;
  type d1 = ^record b :integer; c : array [1..5] of integer end;
  type a1 = record q :integer; q1 : array [1..5] of integer end;
  type d2 = ^integer;
  var y : d2;
  var x: integer;
  var i: integer;
  var p : a1;
  var z : d;
  var z1 : d;
  var m : d3;
  var n : d1;
  procedure q(b: boolean);
    begin b := b = false
    end;
  begin
    new(y);
    y^ := 3;
    x := y^;
    write(y^); {print 3}
    new(m);
    new(n);
    n.b^ := 10;
    n.c[1]^ := 11;
    n.c[4]^ := 11;
    write(n.b^);
    write(n.c[4]^);
    z[1] := 7;
    m[1]^ := z[1];
    q(x > 7);
    i := 1;
    x := 3 * x;
    z[1] := 6;
    z1[1] := 7;
    x := z1[i];
    write(z[1]); {print 6}
    write(x); {print 7}
    write(x * 5) {print 35}
  end
""")
#printSymTab()

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(global $x (mut i32) i32.const 0)
(global $i (mut i32) i32.const 0)
(func $q (param $b i32)
local.get $b
i32.const 0
i32.eq
local.set $b
)
(func $program
i32.const 2
call $new
i32.const 3
i32.const 2
call $set_value
i32.const 2
call $get_value
global.set $x
i32.const 2
call $get_value
call $write
i32.const 0
call $new
i32.const 1
call $new
i32.const 10
i32.

In [24]:
%%writefile test.wat
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(global $x (mut i32) i32.const 0)
(global $i (mut i32) i32.const 0)
(func $q (param $b i32)
local.get $b
i32.const 0
i32.eq
local.set $b
)
(func $program
i32.const 2
call $new
i32.const 3
i32.const 2
call $set_value
i32.const 2
call $get_value
global.set $x
i32.const 2
call $get_value
call $write
i32.const 0
call $new
i32.const 1
call $new
i32.const 10
i32.const 0
i32.const 1
call $set_value1
i32.const 11
i32.const 1
i32.const 1
call $set_value1
i32.const 11
i32.const 4
i32.const 1
call $set_value1
i32.const 0
i32.const 1
call $get_value1
call $write
i32.const 4
i32.const 1
call $get_value1
call $write
i32.const 24
i32.const 7
i32.store
i32.const 24
i32.load
i32.const 0
i32.const 0
call $set_value1
global.get $x
i32.const 7
i32.gt_s
call $q
i32.const 1
global.set $i
i32.const 3
global.get $x
i32.mul
global.set $x
i32.const 24
i32.const 6
i32.store
i32.const 44
i32.const 7
i32.store
global.get $i
i32.const 1
i32.sub
i32.const 4
i32.mul
i32.const 44
i32.add
i32.load
global.set $x
i32.const 24
i32.load
call $write
global.get $x
call $write
global.get $x
i32.const 5
i32.mul
call $write
)
(memory 1)
(start $program)
)

Overwriting test.wat


In [25]:
!wat2wasm test.wat

In [26]:
runpywasm("test.wasm")

3
10
11
6
7
35


In [27]:
print(heap.start) #for showing

size is 4, data: Var(name = y, lev = -3, tp = Pointer(name = , id = 2, lev = , tp = <class 'ST.Int'>)), is_alloc: True, value: 3, nxt: size is 20, data: Var(name = m, lev = -3, tp = Pointer(name = , id = 0, lev = , tp = Array(lower = 1, length = 5, base = <class 'ST.Int'>))), is_alloc: True, value: [7, None, None, None, None], nxt: size is 24, data: Var(name = n, lev = -3, tp = Pointer(name = , id = 1, lev = , tp = Record(fields = [Var(name = b, lev = 1, tp = <class 'ST.Int'>), Var(name = c, lev = 1, tp = Array(lower = 1, length = 5, base = <class 'ST.Int'>))]))), is_alloc: True, value: [10, 11, None, None, 11, None], nxt: None


In [28]:
compileString("""
program p;
  type T = array [3..9] of integer;
  type U = record f: boolean; g: T end;
  var x: U;
  var y: integer;
  begin
    y := y + 3;
    x.f := true;
    x.g[y] := 5;
    write(x.g[y])
  end
""")
#printSymTab()

(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(global $y (mut i32) i32.const 0)
(func $program
global.get $y
i32.const 3
i32.add
global.set $y
i32.const 0
i32.const 1
i32.store
global.get $y
i32.const 3
i32.sub
i32.const 4
i32.mul
i32.const 4
i32.add
i32.const 5
i32.store
global.get $y
i32.const 3
i32.sub
i32.const 4
i32.mul
i32.const 4
i32.add
i32.load
call $write
)
(memory 1)
(start $program)
)


In [29]:
%%writefile test.wat
(module
(import "P0lib" "write" (func $write (param i32)))
(import "P0lib" "writeln" (func $writeln))
(import "P0lib" "read" (func $read (result i32)))
(import "P0lib" "new" (func $new (param i32)))
(import "P0lib" "dispose" (func $dispose (param i32)))
(import "P0lib" "set_value" (func $set_value (param i32) (param i32)))
(import "P0lib" "get_value" (func $get_value (param i32) (result i32)))
(import "P0lib" "set_value1" (func $set_value1 (param i32) (param i32) (param i32)))
(import "P0lib" "get_value1" (func $get_value1 (param i32) (param i32) (result i32)))
(import "P0lib" "set_value2" (func $set_value2 (param i32) (param i32)))
(global $y (mut i32) i32.const 0)
(func $program
global.get $y
i32.const 3
i32.add
global.set $y
i32.const 0
i32.const 1
i32.store
global.get $y
i32.const 3
i32.sub
i32.const 4
i32.mul
i32.const 4
i32.add
i32.const 5
i32.store
global.get $y
i32.const 3
i32.sub
i32.const 4
i32.mul
i32.const 4
i32.add
i32.load
call $write
)
(memory 1)
(start $program)
)

Overwriting test.wat


In [30]:
!wat2wasm test.wat

In [31]:
runpywasm("test.wasm")

5
